# Quantum Pipeline

Define small feature map (angle encoding, ZZFeatureMap).

Define variational ansatz (1–3 layers).

Measure expectation values → feature vector.

Train classical classifier on these features.

In [41]:
import pennylane as qml
from qiskit.visualization import array_to_latex
import pennylane.numpy as np
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit_machine_learning.circuit.library import RawFeatureVector
import matplotlib.pyplot as plt



In [10]:
data = np.load("data/mnist01_pca4.npz")    #from dataprep notebook

X_train, y_train = data["X_train"], data["y_train"]  
X_test, y_test = data["X_test"], data["y_test"]

# x = features/ dimensions --> 4
# y = labels

print("PCA reduced shape xtrain:  ", X_train.shape)
print("PCA reduced shape xtest:  ", X_test.shape)
print("ytrain labels: ", y_train.shape)
print("ytest labels: ",y_test.shape)


PCA reduced shape xtrain:   (12665, 4)
PCA reduced shape xtest:   (2115, 4)
ytrain labels:  (12665,)
ytest labels:  (2115,)


In [ ]:
sample_vec = X_train[0]      #1st of 12665
print("Example feature vector: ", sample_vec)  


Example feature vector:  [-3.57018591  0.12176328  0.03377896  3.41014041]


## Quantum Feature Map

Classical data is transformed into quantum states using pennylane (with select exmaples of processes within Qiskit). For the main pipeline, I will be using PennyLane as it is easier to build custom circuits and I can compare feature maps in a single framework. Pennylane also supports Qiskit backeneds for a future plugin to run on IBM devices. 

<br>

This is a quantum map $\phi(\mathbf{x})$ from classical feature vector $\mathbf{x}$ to quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$ in the Hilbert space, achieved through applying the parameterized unitary operator $\mathcal{U}_{\Phi(\mathbf{x})}$ to the initial state $|0\rangle^{n}$, n being the num of qubits for encoding. This is done so quantum algorithms can use the data. 

Unitary operations are a building block of Qcircuits, where a matrix U is unitary if $$U^\dagger U = U U^\dagger = I$$ 

Some examples are the Pauli X Gate or Hadamard Gate, or in this case, the Rotation Gate $R_y(\theta)$. 
$$
R_y(\theta) = \begin{bmatrix}\cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2)\end{bmatrix}, \quad
R_y^\dagger(\theta) = R_y(-\theta), \quad
R_y(\theta) R_y^\dagger(\theta) = I
$$


All equation .png pictures are sourced from, along with references to their encoding techqniues within descriptions/code: 
https://iraitzm.github.io/qc-handbook/parts/qml/featuremaps.html


### Basis encoding

simpler method of mapping classical data in quantum states, requires data to be composed of ints and maps its binary to the basis state that maps same binary string. So [1,1,1] -> |111⟩. For example: 

<img src="imgs/basis_encoding_function.png" alt="Alt text" width="250"/>

<br>

Because basis encoding needs an input of 0s and 1s (encodess a computational state), we need to convert the PCA features from continuous numbers to ints. 

In [38]:
sample_vec_int = (X_train[0] > 0).astype(int)   # threshold at 0
print(sample_vec_int)

[0 1 1 1]


In [ ]:
#creates pennylane quantum device that simualtes qubits on cpu
dev = qml.device('default.qubit', wires=4)     #qubits = wires
 

@qml.qnode(dev)   #wraps the function, becomes quantum node executable on dev
def feature_map_be(x): 
    #applies X gates where x[i]=1
    qml.BasisEmbedding(features=x, wires=range(4))      #binarize for basis
    return qml.state()


# array_to_latex(array=feature_map_be(sample_vec))
state = feature_map_be(sample_vec_int)     #stores statevector
print(f"Basis encoding: \n {state} \n")

print(qml.draw(feature_map_be, level='device')(sample_vec_int))


Basis encoding: 
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j] 

0: ─╭|Ψ⟩─┤ ╭State
1: ─├|Ψ⟩─┤ ├State
2: ─├|Ψ⟩─┤ ├State
3: ─╰|Ψ⟩─┤ ╰State


We are working with 2**4 = 16 possible states (∣0000⟩, ∣0001⟩, ... ,∣1111⟩). This output shows all complex amplitudes (a raw vector), with all but one zeros. 

(Below I print out the non-zero basis states (there is only the one) in a more readble fashion)

The circuit is depicting a |Ψ⟩ operation (basis embedding) applied jointly across 4 qubits (no gates) preparing a computational basis state. Returns a quantum state vector.

In [44]:
nonzero_indices= np.nonzero(state)[0]   

for idx in nonzero_indices:
    print(f"|{idx:04b}>: amplitude = {state[idx]}")


|0111>: amplitude = (1+0j)


### Angle Encoding

Improves as this method embeds rotation angles into gates so we can introduce numeric data as the rotation angle of our given qubit. This is flexible with the rotation axis (X, Y, Z). I am going to use Z rotations here. For example, x = [x1, x2, ... xn] would be mapped to:

<img src="imgs/angle_encoding_function.png" alt="Alt text" width="250"/>

In [ ]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def feature_map_ae(x):
    qml.AngleEmbedding(features=x, wires=range(4), rotation='Z')     #applies z rotations
    return qml.state()

# X = [2*np.pi, 0, np.pi]
print(qml.draw(feature_map_ae, level='device')(sample_vec))    

0: ──RZ(-3.57)─┤ ╭State
1: ──RZ(0.12)──┤ ├State
2: ──RZ(0.03)──┤ ├State
3: ──RZ(3.41)──┤ ╰State


The middle operation shown on this circuit has a rotation of the value x[i] around the Z axis on the Bloch sphere. For instance, for qubit[0]: 

$$
R_Z(-3.57)\,|0\rangle = e^{-i(-3.57)/2}\,|0\rangle = e^{\,i\frac{3.57}{2}}\,|0\rangle
$$


#### ZZ Feature Map (manual)


For a ZZ Feature Map, PennyLane does not already have a built in function so we have to do it manually using Hadamard gates, along wiht RZ and ZZ rotations for each qubit pair. This just makes it simpler to integrate over the course of this project. I will still show the Qiskit code below too.

The ZZ feature map is a version of the Pauli feature map, which includes entangling gates (ZZ interactions thorugh controlled-phase gates), done after initial data encoding rotations. This is done to allow feature map to capture any potential correlations between features and then maps the data to a more complex hilbert space. 

There are layers of Hadamard gates, single qubit phase rotations, and 2 qubit controlled phase rotations. 

Hadamard Gates

Creates equal superposition of either state within a qubit

$$
H = \tfrac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\[6pt]
1 & -1
\end{pmatrix}
$$

RZ Gates 

Single qubit rotation around Z axis, diagonal gate

$$
R_Z(\phi) 
= \exp\!\left(-i \tfrac{\phi}{2} Z\right) 
= 
\begin{pmatrix}
e^{-i\phi/2} & 0 \\[6pt]
0 & e^{\,i\phi/2}
\end{pmatrix}
$$


CNOT Gates

Flips target qubit if control qubit is 1. 

$$
\text{CNOT} = CX =
\begin{pmatrix}
1 & 0 & 0 & 0 \\[6pt]
0 & 1 & 0 & 0 \\[6pt]
0 & 0 & 0 & 1 \\[6pt]
0 & 0 & 1 & 0
\end{pmatrix}
$$


In [ ]:
def zz_fm(x, wires, reps=1):      #(classical data vector, qubits in use, number of layers/repititions)
    n_qubits = len(wires)

    for _ in range(reps):
        for i in wires: 
            qml.Hadamard(wires=i)            #hadamard gates, uniform superposition of states --> gives room for inference in future entanglement 

        for i, wire in enumerate(wires):     #single qubit phase rotations (encodes classical data to qubit phases)
            qml.RZ(2*x[i], wires=wire)       #RZ rotation, angle = 2x[i] 

        for i in range(n_qubits-1):                    #ZZ entangling layer for pairs of neighboring qubits/wires
            qml.CNOT(wires=[wires[i], wires[i+1]])
            qml.RZ(2*x[i]*x[i+1], wires=wires[i+1])    #performs rotation around Z axis by given amount (angle = 2x[i] * x[i+1]) ENCODES PAIRWISE CORRELATIONS BETWEEN FEATURES x[i] and x[i+1]
            qml.CNOT(wires=[wires[i], wires[i+1]])     #uncomputes entanglement (so final effect is ZZ interaction)



dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def feature_map_zz(x):
    zz_fm(x, wires=range(4), reps=1)
    return qml.state()


print(f"ZZFeatureMap state vector: \n {feature_map_zz(sample_vec)} \n") 
print(qml.draw(feature_map_zz, level='device')(sample_vec))    


ZZFeatureMap state vector: 
 [0.2373154 +0.07862189j 0.11619852+0.2213547j  0.20258228+0.14649376j
 0.14889752+0.20082213j 0.2389986 -0.07334624j 0.22292469+0.11315733j
 0.2499919 -0.00201294j 0.23893279+0.07356034j 0.04083902-0.24664179j
 0.20063923-0.14914388j 0.11327895-0.22286291j 0.1752807 -0.17826013j
 0.23987904-0.07041341j 0.22152198+0.11587929j 0.2499978 +0.00104919j
 0.23801387+0.07648135j] 

0: ──H──RZ(-7.14)─╭●────────────╭●─────────────────────────────────┤ ╭State
1: ──H──RZ(0.24)──╰X──RZ(-0.87)─╰X─╭●───────────╭●─────────────────┤ ├State
2: ──H──RZ(0.07)───────────────────╰X──RZ(0.01)─╰X─╭●───────────╭●─┤ ├State
3: ──H──RZ(6.82)───────────────────────────────────╰X──RZ(0.23)─╰X─┤ ╰State


A CNOT sandwich, which is seen within the multiqubit entanglement layer above, allows for conditional application of a targets rotation, depending on the control qubit. So, in the end, we are maintaining the computational values of the qubit but keeping the phase entanglement of the qubits (it would look the same on the 0,1 level but now hidden quantum phase correlations encode the data)

##### ZZ Feature Map - Qiskit


Switched to Qiskit here because Pennylane does not have ZZFeatureMap. This is just to show other options of achieving the ZZ Feature Maps with a less hands on approach

performs the same rotations in angle encoding but with extra ZZ interaction within dataponits to capture correlating observations to boost representation. 

is an angle encoding PQC (parameterized quantum circuit) to encode features into quantum states

In [22]:
# num_features = 2  #number of qubits/dimensions 
num_features = X_train.shape[1]   #4


feature_map = ZZFeatureMap(num_features)
# feature_map.draw('mpl')
print(f"Depth of qcircuit: {str(feature_map.decompose(reps=10).depth())}.")  
# feature_map.decompose().draw(output='mpl', fold=20)           #issues with plt not sure why
feature_map.decompose().draw(output='text')


Depth of qcircuit: 31.


┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     ├───┤├─────────────┤                                          └───┘»
q_3: ┤ H ├┤ P(2.0*x[3]) ├───────────────────────────────────────────────»
     └───┘└─────────────┘                                               »
«                                                    »
«q_0: ──────────────────────────────────■─────────■──»
«                                       │         │  »
«q_1: ──────────────────────────────────┼────■────┼──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐  │  »
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├──┼──»
«     └──────────────────────────────┘└───┘└───┘┌─┴─┐»
«q_3: ──────────────────────────────────────────┤ X ├»
«                                               └───┘»
«                                               ┌───┐»
«q_0: ───────────────────────────────────────■──┤ H ├»
«                                            │  └───┘»
«q_1: ──────────────────────────────────■────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐  │    │  »
«q_2: ┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├──┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐┌─┴─┐»
«q_3: ┤ P(2.0*(π - x[0])*(π - x[3])) ├─────┤ X ├┤ X ├»
«     └──────────────────────────────┘     └───┘└───┘»
«             ┌─────────────┐                   »
«q_0: ────────┤ P(2.0*x[0]) ├───────────────────»
«             └─────────────┘              ┌───┐»
«q_1: ──────────────────────────────────■──┤ H ├»
«                                       │  └───┘»
«q_2: ──────────────────────────────────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_3: ┤ P(2.0*(π - x[1])*(π - x[3])) ├┤ X ├┤ X ├»
«     └──────────────────────────────┘└───┘└───┘»
«                                                                          »
«q_0: ──────────────────────────────────■──────────────────────────────────»
«             ┌─────────────┐         ┌─┴─┐┌──────────────────────────────┐»
«q_1: ────────┤ P(2.0*x[1]) ├─────────┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├»
«             └─────────────┘         └───┘└────────────┬───┬─────────────┘»
«q_2: ──────────────────────────────────■───────────────┤ H ├──────────────»
«     ┌──────────────────────────────┐┌─┴─┐             ├───┤              »
«q_3: ┤ P(2.0*(π - x[2])*(π - x[3])) ├┤ X ├─────────────┤ H ├──────────────»
«     └──────────────────────────────┘└───┘             └───┘              »
«                                                                        »
«q_0: ───────■─────────■────────────────────────────────────■─────────■──»
«          ┌─┴─┐       │                                    │         │  »
«q_1: ─────┤ X ├───────┼────────────────────────────────────┼────■────┼──»
«     ┌────┴───┴────┐┌─┴─┐┌──────────────────────────────┐┌─┴─┐┌─┴─┐  │  »
«q_2: ┤ P(2.0*x[2]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├──┼──»
«     ├─────────────┤└───┘└──────────────────────────────┘└───┘└───┘┌─┴─┐»
«q_3: ┤ P(2.0*x[3]) ├───────────────────────────────────────────────┤ X ├»
«     └─────────────┘                                               └───┘»
«                                                    »
«q_0: ───────────────────────────────────────■───────»
«                                            │       »
«q_1: ──────────────────────────────────■────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐  │    │  »
«q_2: ┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├──┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐┌─┴─┐»
«q_3: ┤ P(2.0*(π - x[0])*(π - x[3])) ├─────┤ X ├┤ X ├»
«     └──────────────────────────────┘     └───┘└───┘»
«                                   

In [ ]:
# feature_map.decompose().draw('mpl', fold=-1)

### Amplitude Encoding (RFV)

Takes prob amplitude of elements on an array and matches them to a position where they are placed at. Assuming the data is composed of n values (X = [x0, x1, x2, ... xn]), its quantum representation would be: 


<img src="imgs/amplitude_encoding_function.png" alt="Alt text" width="200"/>

In [ ]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def feature_map_ame(feature_vector):
    qml.AmplitudeEmbedding(features=feature_vector, wires=range(4), normalize=True)
    return qml.state()


# X = [1/2, 1/2, 1/2, 1/2]
amp_vec = sample_vec[:4] / np.linalg.norm(sample_vec[:4])
print(feature_map_ame(amp_vec))

ValueError: State must be of length 16; got length 4. Use the 'pad_with' argument for automated padding.

##### RawFeatureVector - Qiskit

Very similar to the pennylane amplitude embedding above

In [23]:
#init and bind RFV to some random state
feature_dimension = num_features
amplitudes = np.array([1,-1,0,0])/np.sqrt(2)

feature_map = RawFeatureVector(feature_dimension=feature_dimension)
feature_map = feature_map.assign_parameters(amplitudes)
print(f"Depth of qcircuit: {str(feature_map.decompose(reps=10).depth())}.")  
feature_map.decompose().draw(output='text')


Depth of qcircuit: 4.


┌───────────────────────────────────┐
q_0: ┤0                                  ├
     │  Initialize(0.70711,-0.70711,0,0) │
q_1: ┤1                                  ├
     └───────────────────────────────────┘

### Pauli Feature Map



This is a manual, hardcoded version of Qiskits PauliFeatureMap. 

In [ ]:
def pauli_feature_map(x, wires): 
    n_qubits=len(wires)

    for i in wires:
        qml.Hadamard(wires=i)

        

##### PauliFeatureMap - Qiskit

In [25]:
feature_map = PauliFeatureMap(3, paulis=["X", "YZ"], entanglement='linear')
# feature_map.decompose().draw('mpl', fold=-1)
feature_map.decompose().draw(output='text')


┌───┐┌───┐┌─────────────┐┌───┐                »
q_0: ┤ H ├┤ H ├┤ P(2.0*x[0]) ├┤ H ├─────────────■──»
     ├───┤├───┤├─────────────┤├───┤┌─────────┐┌─┴─┐»
q_1: ┤ H ├┤ H ├┤ P(2.0*x[1]) ├┤ H ├┤ Rx(π/2) ├┤ X ├»
     ├───┤├───┤├─────────────┤├───┤├─────────┤└───┘»
q_2: ┤ H ├┤ H ├┤ P(2.0*x[2]) ├┤ H ├┤ Rx(π/2) ├─────»
     └───┘└───┘└─────────────┘└───┘└─────────┘     »
«                                             ┌───┐    ┌───┐»
«q_0: ──────────────────────────────────■─────┤ H ├────┤ H ├»
«     ┌──────────────────────────────┐┌─┴─┐┌──┴───┴───┐└───┘»
«q_1: ┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ Rx(-π/2) ├──■──»
«     └──────────────────────────────┘└───┘└──────────┘┌─┴─┐»
«q_2: ─────────────────────────────────────────────────┤ X ├»
«                                                      └───┘»
«             ┌─────────────┐         ┌───┐                                »
«q_0: ────────┤ P(2.0*x[0]) ├─────────┤ H ├────────────────────────────────»
«             └─────────────┘         └───┘   ┌───┐    ┌───┐┌─────────────┐»
«q_1: ──────────────────────────────────■─────┤ H ├────┤ H ├┤ P(2.0*x[1]) ├»
«     ┌──────────────────────────────┐┌─┴─┐┌──┴───┴───┐├───┤└────┬───┬────┘»
«q_2: ┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├┤ Rx(-π/2) ├┤ H ├─────┤ H ├─────»
«     └──────────────────────────────┘└───┘└──────────┘└───┘     └───┘     »
«                                                                          »
«q_0: ───────────────────────────────■─────────────────────────────────────»
«          ┌───┐     ┌─────────┐   ┌─┴─┐   ┌──────────────────────────────┐»
«q_1: ─────┤ H ├─────┤ Rx(π/2) ├───┤ X ├───┤ P(2.0*(π - x[0])*(π - x[1])) ├»
«     ┌────┴───┴────┐└──┬───┬──┘┌──┴───┴──┐└──────────────────────────────┘»
«q_2: ┤ P(2.0*x[2]) ├───┤ H ├───┤ Rx(π/2) ├────────────────────────────────»
«     └─────────────┘   └───┘   └─────────┘                                »
«                                                                            
«q_0: ──■────────────────────────────────────────────────────────────────────
«     ┌─┴─┐┌──────────┐                                                      
«q_1: ┤ X ├┤ Rx(-π/2) ├──■────────────────────────────────────■──────────────
«     └───┘└──────────┘┌─┴─┐┌──────────────────────────────┐┌─┴─┐┌──────────┐
«q_2: ─────────────────┤ X ├┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├┤ Rx(-π/2) ├
«                      └───┘└──────────────────────────────┘└───┘└──────────┘